In [1]:
# For CPU version
!pip install faiss-cpu
# For GPU version
!pip install faiss-gpu
!pip install sentence_transformers
!pip install pytrec_eval
!pip install ir_datasets
!pip install cherche --upgrade -q
!pip install nltk



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylin

In [2]:
!pip install ir_datasets python-terrier


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-terrier: filename=python_terrier-0.10.1-py3-none-any.whl size=117255 sha256=12b234bc1c716009e270312251e5893236f64f6c56373f2deddc121c2bfba735
  Stored in directory: /root/.cache/pip/wheels/58/88/6d/fd8a

In [13]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import faiss
import torch
from sentence_transformers import SentenceTransformer
import ir_datasets
import numpy as np
import pytrec_eval
import ir_datasets
import nltk
import nltk
from nltk.corpus import wordnet as wn
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

import pyterrier as pt

if not pt.started():
    pt.init()



# Charger le dataset scidocs de BEIR
dataset = ir_datasets.load("vaswani")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



In [4]:
# Extract the document generator
def document_generator():
    for doc in dataset.docs_iter():
        yield {
            'docno': doc.doc_id,
            'text': doc.text
        }


In [5]:
# Define the path where the index will be stored
index_path = './msmarco_index'

# Create the indexer
indexer = pt.IterDictIndexer(index_path, meta=['docno', 'text'], meta_lengths=[20, 4096])

# Index the data
index_ref = indexer.index(document_generator())


[INFO] [starting] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz
[INFO] [finished] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: [00:00] [2.13MB] [2.21MB/s]


In [20]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def preprocess_query(query):
    # Remove problematic characters
    query = re.sub(r"[\"']", "", query)
    # Replace any other problematic characters with a space
    query = re.sub(r"[^\w\s]", " ", query)
    # Trim and ensure proper format
    query = query.strip()
    return query

# Load the queries
queries = [query.text for query in dataset.queries_iter()]
def get_synonyms(word):
    synonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace('_', ' '))
    return synonyms

def expand_query(query):
    words = query.split()
    expanded_queries = set()
    expanded_queries.add(query)

    for word in words:
        synonyms = get_synonyms(word)
        for synonym in synonyms:
            new_query = query.replace(word, synonym)
            expanded_queries.add(new_query)

    return list(expanded_queries)

def treat_querie(text):

  expanded_queries = expand_query(text)
  final_req=[]
  for eq in expanded_queries:
      final_req.append(eq)
  tokens = set()
  for sentence in final_req:
      words = word_tokenize(sentence)
      tokens.update(words)  # Add tokens to the set
      sentence = ' '.join(tokens)
      return sentence

queries = [preprocess_query(treat_querie(query.text)) for query in dataset.queries_iter()]


queries[10]



bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")





In [21]:
run = {}

# Perform retrieval and create the run dictionary
for query in dataset.queries_iter():
    queryid = query.query_id
    querytext = preprocess_query(query.text)
    results = bm25.search(preprocess_query(treat_querie(querytext)))
    run[queryid] = {row.docno: rank + 1 for rank, row in enumerate(results.itertuples())}

# Print or use the run dictionary
print(run)

{'1': {'8172': 1, '9881': 2, '5502': 3, '1502': 4, '9859': 5, '4871': 6, '4817': 7, '8276': 8, '7234': 9, '7923': 10, '720': 11, '6276': 12, '10652': 13, '8582': 14, '8258': 15, '9588': 16, '9591': 17, '8150': 18, '4463': 19, '8565': 20, '2236': 21, '7735': 22, '11212': 23, '2800': 24, '2224': 25, '7230': 26, '6579': 27, '4147': 28, '3693': 29, '4827': 30, '6824': 31, '1756': 32, '3994': 33, '3082': 34, '9295': 35, '6275': 36, '3885': 37, '8319': 38, '3489': 39, '9350': 40, '3010': 41, '3098': 42, '2487': 43, '1879': 44, '8061': 45, '8148': 46, '1897': 47, '719': 48, '5039': 49, '2940': 50, '444': 51, '4533': 52, '2005': 53, '26': 54, '2367': 55, '7734': 56, '5493': 57, '3115': 58, '8883': 59, '5912': 60, '6823': 61, '5145': 62, '3064': 63, '9679': 64, '265': 65, '10178': 66, '8259': 67, '9890': 68, '3191': 69, '3221': 70, '3246': 71, '6635': 72, '1889': 73, '4361': 74, '5144': 75, '4572': 76, '7831': 77, '7941': 78, '4256': 79, '7025': 80, '152': 81, '3068': 82, '9304': 83, '6393': 84

In [22]:
import pytrec_eval
import pandas as pd

qrel = {k: {kk:int(vv) for kk,vv in v[['doc_id','relevance']].values} for k, v in pd.DataFrame(dataset.qrels_iter()).groupby('query_id')[['doc_id','relevance']]}

evaluator = pytrec_eval.RelevanceEvaluator(qrel, {'map', 'ndcg_cut' , 'P.10'})

evaluation_results = evaluator.evaluate(run)

evaluation_df = pd.DataFrame(evaluation_results).T
mean_scores = evaluation_df.mean()

print(mean_scores)

map              0.011797
P_10             0.002151
ndcg_cut_5       0.003647
ndcg_cut_10      0.002367
ndcg_cut_15      0.002395
ndcg_cut_20      0.002494
ndcg_cut_30      0.003699
ndcg_cut_100     0.008586
ndcg_cut_200     0.011671
ndcg_cut_500     0.033241
ndcg_cut_1000    0.257332
dtype: float64
